#### Save the E-values of class_III and class_IV from hmmsearch into a dictionary

In [15]:
# Create an empty dictionary to store sequence IDs and their E-values
# Class_III
class_III_dict = {}

# Open the sequence_evalues.txt file and read each line
with open('../data/class3_LanKC_hmmsearch_evalues.txt', 'r') as f:
    for line in f:
        # Split each line by space to separate the sequence ID and E-value
        seq_id, evalue = line.strip().split()
        # Add the sequence ID and E-value to the dictionary
        class_III_dict[seq_id.split('/')[0]] = float(evalue)

# Class_IV
class_IV_dict = {}

# Open the sequence_evalues.txt file and read each line
with open('../data/class4_LanL_hmmsearch_evalues.txt', 'r') as f:
    for line in f:
        # Split each line by space to separate the sequence ID and E-value
        seq_id, evalue = line.strip().split()
        # Add the sequence ID and E-value to the dictionary
        class_IV_dict[seq_id.split('/')[0]] = float(evalue)


#### Cyclase classification

In [16]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/RODEO_lanthipeptides/main_co_occur_all.csv')

# Create a new column for classification and set default value as 'unclassified'
df['Classification'] = 'unclassified'

# List of unique query IDs
unique_query_ids = df['Query'].unique()

for query_id in unique_query_ids:
    query_rows = df[df['Query'] == query_id]
            
    # Check for class_I criteria
    if any(query_rows['Protein_acc'].isin(query_rows['Query'])):
        if any(query_rows['PfamID1'] == 'PF04738') or any(query_rows['PfamID2'] == 'PF04738') or any(query_rows['PfamID3'] == 'PF04738'):
            if any(query_rows['PfamID1'] == 'PF14028') or any(query_rows['PfamID2'] == 'PF14028') or any(query_rows['PfamID3'] == 'PF14028'):
                df.loc[df['Query'] == query_id, 'Classification'] = 'class_I'
                continue
                
    # Check for class_II criteria
    if any(query_rows['Protein_acc'].isin(query_rows['Query'])):
        if any(query_rows['PfamID1'] == 'PF13575') or any(query_rows['PfamID2'] == 'PF13575') or any(query_rows['PfamID3'] == 'PF13575'):
            df.loc[df['Query'] == query_id, 'Classification'] = 'class_II'
            continue

    # Check for class_III_IV criteria
    class_III_IV_rows = query_rows[((query_rows['PfamID1'] == 'PF05147') & (query_rows['PfamID2'] == 'PF00069')) | 
                                   ((query_rows['PfamID1'] == 'PF05147') & (query_rows['PfamID3'] == 'PF00069')) | 
                                   ((query_rows['PfamID2'] == 'PF05147') & (query_rows['PfamID1'] == 'PF00069')) | 
                                   ((query_rows['PfamID2'] == 'PF05147') & (query_rows['PfamID3'] == 'PF00069')) | 
                                   ((query_rows['PfamID3'] == 'PF05147') & (query_rows['PfamID1'] == 'PF00069')) | 
                                   ((query_rows['PfamID3'] == 'PF05147') & (query_rows['PfamID2'] == 'PF00069'))]
    if not class_III_IV_rows.empty:
        # Extract sequence ID and its corresponding E-value
        for _, row in class_III_IV_rows.iterrows():
            seq_id = row['Query']
            # print(seq_id)
        
            if seq_id in class_III_dict and seq_id not in class_IV_dict:
                df.loc[df['Query'] == seq_id, 'Classification'] = 'class_III'
            
            elif seq_id not in class_III_dict and seq_id in class_IV_dict:
                df.loc[df['Query'] == seq_id, 'Classification'] = 'class_IV'
        
            elif seq_id in class_III_dict and seq_id in class_IV_dict:
                if class_III_dict[seq_id] < class_IV_dict[seq_id]:
                    df.loc[df['Query'] == seq_id, 'Classification'] = 'class_III'
                else:
                    df.loc[df['Query'] == seq_id, 'Classification'] = 'class_IV'

# Save the modified dataframe to a new CSV file
df.to_csv('../data/RODEO_lanthipeptides/main_co_occur_all_classification.csv', index=False)


WP_204165024.1
WP_067010791.1
WP_028636691.1
WP_190111090.1
WP_225662071.1
WP_307540222.1
WP_215074734.1
WP_282319562.1
WP_249913016.1
WP_253429555.1
WP_145151972.1
WP_190189269.1
WP_307415189.1
WP_250966801.1
WP_150365363.1
WP_264926919.1
WP_246151153.1
WP_188277711.1
WP_182444875.1
WP_228647165.1
WP_160712809.1
WP_283469482.1
WP_251509807.1
WP_199272541.1
WP_127038628.1
WP_261712768.1
WP_229352041.1
WP_161289037.1
WP_189849960.1
WP_189648376.1
WP_141721336.1
WP_184737541.1
WP_253498212.1
WP_184932808.1
WP_189970167.1
WP_145333979.1
WP_208630522.1
WP_161244321.1
WP_190025991.1
WP_157855733.1
WP_190014676.1
WP_189833869.1
WP_221790666.1
WP_171079204.1
WP_181442350.1
WP_306325137.1
WP_281361620.1
WP_207400631.1
WP_188270229.1
WP_301247386.1
WP_227725671.1
WP_114040576.1
WP_266356385.1
WP_190105690.1
WP_224323679.1
WP_218040148.1
WP_067167918.1
WP_308013239.1
WP_167927409.1
WP_285527400.1
WP_306311378.1
WP_207918587.1
WP_184611967.1
WP_185054200.1
WP_167160948.1
WP_187752115.1
WP_1498479

#### Find the directories of predicted RiPP precursor peptides

In [17]:
import pandas as pd

# Open the file for reading
with open('../data/RODEO_lanthipeptides/unclassified_cyc/unclassified_cyclases.txt', 'r') as file:
    # Read each line, strip any whitespace, and store in a list
    id_list = [line.strip() for line in file]

# Load the CSV file
df = pd.read_csv('../data/RODEO_lanthipeptides/main_co_occur_all_classification.csv')

# List of unique query IDs
unique_query_ids = df['Query'].unique()

id_to_file = {}
for query_id in unique_query_ids:
    if query_id in id_list[0:1000]:
        df.loc[df['Query'] == query_id, 'File'] = '1254_RODEO_Results/unclassified_cyclase1'
    elif query_id in id_list[1000:2000]:
        df.loc[df['Query'] == query_id, 'File'] = '1278_RODEO_Results/unclassified_cyclase2_2'
    elif query_id in id_list[2000:3000]:
        df.loc[df['Query'] == query_id, 'File'] = '1279_RODEO_Results/unclassified_cyclase3_2'
    elif query_id in id_list[3000:4000]:
        df.loc[df['Query'] == query_id, 'File'] = '1280_RODEO_Results/unclassified_cyclase4_2'
    elif query_id in id_list[4000:5000]:
        df.loc[df['Query'] == query_id, 'File'] = '1281_RODEO_Results/unclassified_cyclase5'
    elif query_id in id_list[5000:6000]:
        df.loc[df['Query'] == query_id, 'File'] = '1297_RODEO_Results/unclassified_cyclase6'
    elif query_id in id_list[6000:7000]:
        df.loc[df['Query'] == query_id, 'File'] = '1298_RODEO_Results/unclassified_cyclase7'
    elif query_id in id_list[7000:8000]:
        df.loc[df['Query'] == query_id, 'File'] = '1299_RODEO_Results/unclassified_cyclase8'
    else:
        df.loc[df['Query'] == query_id, 'File'] = '1300_RODEO_Results/unclassified_cyclase9'

# Save the modified dataframe to a new CSV file
df.to_csv('../data/RODEO_lanthipeptides/main_co_occur_all_classification_file.csv', index=False)

#### Organize all potential RiPP precursor peptides into one file

In [22]:
import pandas as pd

def read_lanthi(file_path, class_no, id):
    csv_map = {
        'class_I': 'lanthi1/lanthi1_features.csv',
        'class_II': 'lanthi2/lanthi2_features.csv',
        'class_III': 'lanthi3/lanthi3_features.csv',
        'class_IV': 'lanthi4/lanthi4_features.csv',
    }

    if class_no in csv_map:
        if file_path not in dfs_cache:
            dfs_cache[file_path] = {}
        if class_no not in dfs_cache[file_path]:
            dfs_cache[file_path][class_no] = pd.read_csv(f'../data/RODEO_lanthipeptides/{file_path}/{csv_map[class_no]}')
        df = dfs_cache[file_path][class_no]
        filtered_df = df[(df['Accession_id'] == id) & (df['Classification'] == 1)].iloc[:, :10]
        filtered_df['Lanthipeptide_class'] = class_no  # Add the new column
        return filtered_df
    return None

# Load the CSV file
df = pd.read_csv('../data/RODEO_lanthipeptides/main_co_occur_all_classification_file.csv')
unique_pairs = df[['Query', 'Classification', 'File']].drop_duplicates()

dfs_cache = {}  # Dictionary to cache dataframes
output_dfs = []

for _, row in unique_pairs.iterrows():
    query_id = row['Query']
    classification = row['Classification']
    file_path = row['File']
    filtered_df = read_lanthi(file_path, classification, query_id)
    if filtered_df is not None:
        output_dfs.append(filtered_df)

final_df = pd.concat(output_dfs, axis=0)
final_df.to_csv('../data/RODEO_lanthipeptides/new_RiPPs_from_unclassified.csv', index=False)


#### Save the RiPP precursor peptides into fasta files

In [1]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/RODEO_lanthipeptides/new_RiPPs_from_unclassified.csv')

# Define a function to generate a FASTA format string
def to_fasta(row):
    header = f">{row['Accession_id']}/{row['Start']}-{row['End']}"
    sequence = row['Leader'] + row['Core']
    return f"{header}\n{sequence}\n"

# Loop through each class and generate the corresponding FASTA file
for lanthi_class in ['class_I', 'class_II', 'class_III', 'class_IV']:
    # Filter rows based on the Lanthipeptide_class
    filtered_df = df[df['Lanthipeptide_class'] == lanthi_class].copy()
    
    # Compute the length of the sequence for each row
    filtered_df['sequence_length'] = filtered_df.apply(lambda row: len(row['Leader'] + row['Core']), axis=1)
    
    # Filter rows based on the sequence length
    filtered_df = filtered_df[(filtered_df['sequence_length'] >= 30) & (filtered_df['sequence_length'] <= 120)]
    
    with open(f'../data/lanthipeptide/{lanthi_class}-precursors_peptide_unclassified.fasta', 'w') as fasta_file:
        # Generate FASTA format strings and write to file
        fasta_file.writelines(filtered_df.apply(to_fasta, axis=1))


#### Save the RiPP precursor peptides into fasta files (separate leader and core)

In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/RODEO_lanthipeptides/new_RiPPs_from_unclassified.csv')

# Define a function to generate a FASTA format string
def to_fasta_leader(row):
    header = f">{row['Accession_id']}/{row['Start']}-{row['End']}"
    sequence = row['Leader']
    return f"{header}\n{sequence}\n"

def to_fasta_core(row):
    header = f">{row['Accession_id']}/{row['Start']}-{row['End']}"
    sequence = row['Core']
    return f"{header}\n{sequence}\n"

# Loop through each class and generate the corresponding FASTA file
for lanthi_class in ['class_I', 'class_II', 'class_III', 'class_IV']:
    # Filter rows based on the Lanthipeptide_class
    filtered_df = df[df['Lanthipeptide_class'] == lanthi_class].copy()
    
    # Compute the length of the sequence for each row
    filtered_df['leader_sequence_length'] = filtered_df.apply(lambda row: len(row['Leader']), axis=1)

    # Compute the length of the sequence for each row
    filtered_df['core_sequence_length'] = filtered_df.apply(lambda row: len(row['Leader']), axis=1)
    
    # Filter rows based on the sequence length
    filtered_df = filtered_df[filtered_df['leader_sequence_length'] <= 120]
    filtered_df = filtered_df[filtered_df['core_sequence_length'] <= 120]
    
    with open(f'../data/lanthipeptide/{lanthi_class}-precursors_peptide_leader_unclassified.fasta', 'w') as fasta_file:
        # Generate FASTA format strings and write to file
        fasta_file.writelines(filtered_df.apply(to_fasta_leader, axis=1))

    with open(f'../data/lanthipeptide/{lanthi_class}-precursors_peptide_core_unclassified.fasta', 'w') as fasta_file:
        # Generate FASTA format strings and write to file
        fasta_file.writelines(filtered_df.apply(to_fasta_core, axis=1))